<a href="https://colab.research.google.com/github/dolmani38/QA/blob/main/Train_korQuAD1.0_0516.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국어 Albert Tokenizer 학습!

In [1]:

if True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


## 참조

https://colab.research.google.com/github/parmarsuraj99/suraj-parmar/blob/master/_notebooks/2020-05-02-SanskritALBERT.ipynb#scrollTo=VNAOMXjpMHZD


In [2]:
import os
import gc
import glob
import torch
import pickle
import joblib
from tqdm.auto import tqdm

In [3]:
!git clone https://github.com/huggingface/transformers
!pip install transformers/.
!pip install sentencepiece==0.1.95
!pip install datasets==1.6.2

Cloning into 'transformers'...
remote: Enumerating objects: 72664, done.
remote: Counting objects: 100% (702/702), done.
remote: Compressing objects: 100% (375/375), done.
remote: Total 72664 (delta 402), reused 496 (delta 287), pack-reused 71962
Receiving objects: 100% (72664/72664), 56.17 MiB | 26.85 MiB/s, done.
Resolving deltas: 100% (51530/51530), done.
Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 4.2MB/s 
     |████████████████████████████████| 901kB 32.3MB/s 
  Created wheel for transformers: filename=transformers-4.7.0.dev0-cp37-none-any.whl size=2259809 sha256=b3393a1f7dfd6214f6b775a1ad292efa6a4b2da94954ea168efbdf64e7bda812
  Stored in directory: /tmp/pip-ephem-wheel-cache-2fk6hb2y/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
     |████████████████████████████████| 1.2MB 4.

## mecab 설치 참조

https://somjang.tistory.com/entry/Google-Colab%EC%97%90%EC%84%9C-Mecab-koMecab-ko-dic-%EC%89%BD%EA%B2%8C-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0

In [18]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [19]:
!bash ./Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 460kB 38.5MB/s 
     |████████████████████████████████| 92kB 10.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-05-16 00:27:56--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::6b17:d1f5, 2406:da00:ff00::22cd:e0db, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=67OlFPehmTqro5oAkWE3QxK%2FEUs%3D&Expires=1621126677&AWSAccessKeyId=AKIA6KOSE3

# 한국어 Tokenizer 학습

## dataset 만들기

### 네이버 뉴스.... 영화 평가 댓글... 에서 

In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:

# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    #txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    #txt = txt.replace('.','. ')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()


In [23]:
%%time

import re
import pandas as pd
import sys

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/summary/korean_news_corpus.csv')
# 검사...
pattens = ["[34569][0-9]{3}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}",
           "[0-9]{2,3}[\:\s\;.\;,\;-;)][0-9]{3,4}[\:\s\;.\;,\;-][0-9]{4}",
           "[0-9]{1}[0-9]{1}[\W]?[0-1]{1}[0-9]{1}[\W]?[0-3]{1}[\W]?[0-9]{1}[\W]?[1-4]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}",
           "[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{3}[\:\s\;.\;,\;-]([0-9]{5,6}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{5}|[0-9]{2,3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{7}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{4,6}[\:\s\;.\;,\;-][0-9]|[0-9]{5}[\:\s\;.\;,\;-][0-9]{3}[\:\s\;.\;,\;-][0-9]{2}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{5}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{4}[\:\s\;.\;,\;-][0-9]{4}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{7})|[0-9]{4}[\:\s\;.\;,\;-]([0-9]{3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{6}[\:\s\;.\;,\;-][0-9])|[0-9]{5}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{5,6}"
           ]

filters = []
for p in pattens:
    filters.append(re.compile(p))

sentences = []
df = df.dropna(axis=0)
cnt = df['contents'].count()
print('Total row count:',cnt)
i=0
for raw_text in df['contents']:
    i=i+1
    try:
        if i%100 == 0:
            percent = ("{0:.2f}").format(100 * (i / float(cnt)))
            print(f'\r {percent}% {i}/{str(cnt)}', end="", flush=True)

        docs = nltk.sent_tokenize(clean_text(raw_text))
        for txt in docs:
            if txt.find('▶') > -1 or txt.find('@') > -1 or txt.find('ⓒ') > -1: 
                pass
            else:
                txt = txt.strip()
                if any(chr.isdigit() for chr in txt) :
                    pass
                else:
                    sentences.append(txt)
    except KeyboardInterrupt as ki:
        raise ki        
    except:
        pass #print("Unexpected error:", sys.exc_info()[0])

print('')

Total row count: 140536
 99.97% 140500/140536
CPU times: user 3min 19s, sys: 4.44 s, total: 3min 24s
Wall time: 3min 25s


In [24]:
len(sentences)

2967202

In [25]:
sentences[:10]

['연합뉴스 문재인 대통령이 변창흠 국토교통부 장관의 사의 표명을 사실상 수용했다.',
 '문 대통령은 변 장관의 사의 표명에 책임지는 모습을 보일 수밖에 없다고 밝혔다.',
 '이는 문 대통령이 사실상 변 장관의 사의를 수용한 것으로 해석된다.',
 '이에 앞서 변창흠 국토교통부 장관은 LH 땅 투기 의혹 사건과 관련한 책임론에 대해 “자리에 연연하지 않는다”면서 “(청와대의) 결정에 따르겠다”고 말했다.',
 '변 장관은 이날 국회 국토교통위원회 전체회의에 참석해 “LH 사태로 국민들이 걱정하는 부분을 해소할 수 있게 최대한 대안을 만들고 LH가 근본적으로 다시 태어날 수 있도록 책임지고 추진하겠다”고 언급하고 “그 역할이 충분하다고 평가되지 못했을 때 언제든지 자리에 연연하지 않고 결정에 따르겠다”고 말했다.',
 'kr ) 무단 전재 및 재배포 금지',
 'kr ) 무단 전재 및 재배포 금지',
 'kr ) 무단 전재 및 재배포 금지',
 '인재 양성·소외 계층 지원 등 계획 “부친 질병·가난 악순환 끊기 원해 국내 최고 넘어 세계적 병원 되길 정몽구 현대차그룹 명예회장.',
 '그는 “질병과 가난이 악순환되는 고리를 끊기 위해 아산재단과 서울아산병원을 설립했던 아버님의 뜻을 이어 우리 사회의 어려운 이웃을 돕는 데 보탬이 되기를 바란다”고 기부 취지를 밝혔다.']

In [26]:
%%time

import re
import sys
import io

#텍스트 정제(전처리)
def cleanText(readData):
    #텍스트에 포함되어 있는 특수 문자 제거
    text = re.sub('[-=+#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》◆◇●🎧○▲\t―△━▷]', '', readData)
    return text

c_sentences = []
for sentence in sentences:
    s = cleanText(sentence)
    c = len(s.split())
    if c >= 3 and c < 10 and s.find('재배포') < 0 and s.find('기자') < 0  and s.find('유투브') < 0 and s.find('www') < 0 and s.find('com') < 0 and s.find('접속하기') < 0 and s.find('http') < 0 and s.find('뉴스') < 0 and s.find('일보') < 0 :
        if s.endswith(('다','요')):
            c_sentences.append(s.strip())    

CPU times: user 12.8 s, sys: 194 ms, total: 13 s
Wall time: 13 s


In [27]:
len(c_sentences)

867766

In [28]:
# 한국어 영화 리뷰를 Load.
ds = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",sep='\t')
for row in ds.iterrows():
    doc_id = row[1][0]
    doc_cont = str(row[1][1])
    c_sentences.append(doc_cont)

In [29]:
len(c_sentences)

1017766

In [30]:
import random

random.shuffle(c_sentences)

### 한국어... 교착어 처리...

In [31]:
%%time
# mecab for window는 아래 코드 사용
from konlpy.tag import Mecab  # install mecab for window: https://hong-yp-ml-records.tistory.com/91
mecab_tokenizer = Mecab().morphs
print('mecab check :', mecab_tokenizer('어릴때보고 지금다시봐도 재밌어요ㅋㅋ'))

ko_sentences = []

for sentence in c_sentences:
    # 문장단위 mecab 적용
    morph_sentence= mecab_tokenizer(sentence)
    # 문장단위 저장
    ko_sentences.append( (' '.join(morph_sentence)).strip())
        
num_word_list = [len(sentence.split()) for sentence in ko_sentences]
print('\n힌국어 코퍼스 평균/총 단어 갯수 : %.1f / %d' % (sum(num_word_list)/len(num_word_list), sum(num_word_list)))       

mecab check : ['어릴', '때', '보', '고', '지금', '다시', '봐도', '재밌', '어요', 'ㅋㅋ']

힌국어 코퍼스 평균/총 단어 갯수 : 13.7 / 13929565
CPU times: user 1min 23s, sys: 849 ms, total: 1min 24s
Wall time: 1min 24s


In [32]:
%%time
# subword 학습을 위해 문장만 따로 저장
with open('/content/drive/MyDrive/Tokenizer_train/data/train_tokenizer.txt', 'w', encoding='utf-8') as f:
    for line in ko_sentences:
        f.write(line+'\n')

f.close()

CPU times: user 575 ms, sys: 115 ms, total: 691 ms
Wall time: 2.14 s


### CNN/Daily main Dataset에서 영어... 문장 

In [22]:

import tensorflow_datasets as tfds
train_data, test_data = tfds.load(name="cnn_dailymail",split=(tfds.Split.TRAIN,tfds.Split.TEST),with_info=True)

KeyboardInterrupt: ignored

In [ ]:
%%time
en_sentences = []
iterator = iter(test_data[0])
for data in iterator:
    article = nltk.sent_tokenize(data['article'].numpy().decode('UTF-8'))
    for sent in article:    
        en_sentences.append(sent)
        

In [ ]:
num_word_list = [len(sentence.split()) for sentence in en_sentences]
print('\n영어 코퍼스 평균/총 단어 갯수 : %.1f / %d' % (sum(num_word_list)/len(num_word_list), sum(num_word_list)))

In [28]:
%%time
# subword 학습을 위해 문장만 따로 저장
with open('/content/drive/MyDrive/Tokenizer_train/data/train_tokenizer.txt', 'w', encoding='utf-8') as f:
    for line in ko_sentences:
        f.write(line+'\n')

f.close()

CPU times: user 288 ms, sys: 40.3 ms, total: 328 ms
Wall time: 1.35 s


# 여기서부터 다시

In [8]:
%%time
import sentencepiece as spm
import os
# spm_train --input=data/train_tokenizer.txt  --model_prefix=sentencepiece/sp --vocab_size=32000 character_coverage=1.0 --model_type="unigram"

input_file = '/content/drive/MyDrive/Tokenizer_train/data/train_tokenizer.txt'
vocab_size = 30015

sp_model_root='/content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model'
if not os.path.isdir(sp_model_root):
    os.mkdir(sp_model_root)
sp_model_name = 'spiece'
sp_model_path = os.path.join(sp_model_root, sp_model_name)
model_type = 'unigram'  # 학습할 모델 선택, unigram이 더 성능이 좋음'bpe'
character_coverage  = 1.0  # 전체를 cover 하기 위해, default=0.9995
user_defined_symbols = '[PAD],[UNK],[CLS],[SEP],[MASK],[BOS],[EOS],[UNK0],[UNK1],[UNK2],[UNK3],[UNK4],[UNK5],[UNK6],[UNK7],[UNK8],[UNK9],[unused0],[unused1],[unused2],[unused3],[unused4],[unused5],[unused6],[unused7],[unused8],[unused9],[unused10],[unused11],[unused12],[unused13],[unused14],[unused15],[unused16],[unused17],[unused18],[unused19],[unused20],[unused21],[unused22],[unused23],[unused24],[unused25],[unused26],[unused27],[unused28],[unused29],[unused30],[unused31],[unused32],[unused33],[unused34],[unused35],[unused36],[unused37],[unused38],[unused39],[unused40],[unused41],[unused42],[unused43],[unused44],[unused45],[unused46],[unused47],[unused48],[unused49],[unused50],[unused51],[unused52],[unused53],[unused54],[unused55],[unused56],[unused57],[unused58],[unused59],[unused60],[unused61],[unused62],[unused63],[unused64],[unused65],[unused66],[unused67],[unused68],[unused69],[unused70],[unused71],[unused72],[unused73],[unused74],[unused75],[unused76],[unused77],[unused78],[unused79],[unused80],[unused81],[unused82],[unused83],[unused84],[unused85],[unused86],[unused87],[unused88],[unused89],[unused90],[unused91],[unused92],[unused93],[unused94],[unused95],[unused96],[unused97],[unused98],[unused99]'

input_argument = '--input=%s --model_prefix=%s --vocab_size=%s --user_defined_symbols=%s --model_type=%s --character_coverage=%s'
cmd = input_argument%(input_file, sp_model_path, vocab_size,user_defined_symbols, model_type, character_coverage)

spm.SentencePieceTrainer.Train(cmd)
print('train done')

train done
CPU times: user 33.2 s, sys: 667 ms, total: 33.9 s
Wall time: 28.6 s


In [9]:
## 1) define special tokens
user_defined_symbols = ['[BOS]','[EOS]','[UNK0]','[UNK1]','[UNK2]','[UNK3]','[UNK4]','[UNK5]','[UNK6]','[UNK7]','[UNK8]','[UNK9]']
unused_token_num = 200
unused_list = ['[unused{}]'.format(n) for n in range(unused_token_num)]
user_defined_symbols = user_defined_symbols + unused_list

print(user_defined_symbols)

['[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31]', '[unused32]', '[unused33]', '[unused34]', '[unused35]', '[unused36]', '[unused37]', '[unused38]', '[unused39]', '[unused40]', '[unused41]', '[unused42]', '[unused43]', '[unused44]', '[unused45]', '[unused46]', '[unused47]', '[unused48]', '[unused49]', '[unused50]', '[unused51]', '[unused52]', '[unused53]', '[unused54]', '[unused55]', '[unused56]', '[unused57]', '[unused58]', '[unused59]', '[unused60]', '[unused61]', '[unused62]', '[unused6

In [19]:
#'/content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model'

from transformers import BertTokenizer, AlbertTokenizer

albet_tokenizer_model = '/content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model'

tokenizer = AlbertTokenizer.from_pretrained(albet_tokenizer_model)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
op = tokenizer.encode("나는 오늘 학교에 간다.")
print(op)
tokenizer.decode(op)


[5, 157, 491, 395, 740, 1614, 1723, 11640, 6]


'[CLS] 나는 오늘 학교에 간다.[SEP]'

In [21]:
# tokenizer에 special token 추가
special_tokens_dict = {'additional_special_tokens': user_defined_symbols}
tokenizer.add_special_tokens(special_tokens_dict)

# check tokenizer vocab with special tokens
print('check special tokens : %s'%tokenizer.all_special_tokens[:20])

check special tokens : ['[CLS]', '[SEP]', '<unk>', '<pad>', '[MASK]', '[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]', '[unused0]', '[unused1]', '[unused2]']


In [22]:
# save tokenizer model with special tokens
tokenizer.save_pretrained(albet_tokenizer_model+'_special2')

('/content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model_special2/tokenizer_config.json',
 '/content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model_special2/special_tokens_map.json',
 '/content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model_special2/spiece.model',
 '/content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model_special2/added_tokens.json')

In [23]:
tokenizer = AlbertTokenizer.from_pretrained(albet_tokenizer_model+'_special2')

In [24]:
op = tokenizer("나는 오늘 학교에 간다.", return_tensors="pt")
print(op)
print("Tokens (str)      : {}".format([tokenizer.convert_ids_to_tokens(s) for s in op['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(op['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(op['attention_mask'].tolist()[0]))


{'input_ids': tensor([[    5,   157,   491,   395,   740,  1614,  1723, 11640,     6]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Tokens (str)      : ['[CLS]', '▁나', '는', '▁오늘', '▁학교', '에', '▁간다', '.', '[SEP]']
Tokens (int)      : [5, 157, 491, 395, 740, 1614, 1723, 11640, 6]
Tokens (attn_mask): [1, 1, 1, 1, 1, 1, 1, 1, 1]



In [25]:
#Checking vocabulary size
vocab_size=tokenizer.vocab_size ; vocab_size

30015

In [26]:
import json

config = {
    "_name_or_path": "dolmani38/albert-xlarge-kor-v1",
    "architectures": [
        "AlbertModel"
    ],
    "attention_probs_dropout_prob": 0,
    "bos_token_id": 2,
    "classifier_dropout_prob": 0.1,
    "down_scale_factor": 1,
    "embedding_size": 128,
    "eos_token_id": 3,
    "gap_size": 0,
    "hidden_act": "gelu_new",
    "hidden_dropout_prob": 0,
    "hidden_size": 2048,
    "initializer_range": 0.02,
    "inner_group_num": 1,
    "intermediate_size": 8192,
    "layer_norm_eps": 1e-12,
    "max_position_embeddings": 512,
    "model_type": "albert",
    "net_structure_type": 0,
    "num_attention_heads": 16,
    "num_hidden_groups": 1,
    "num_hidden_layers": 24,
    "num_memory_blocks": 0,
    "pad_token_id": 0,
    "position_embedding_type": "absolute",
    "type_vocab_size": 2,
	"vocab_size": vocab_size
}
with open(albet_tokenizer_model + "_special2/config.json", 'w') as fp:
    json.dump(config, fp)


#Configuration for tokenizer.
#Note: I set do_lower_case: False, and keep_accents:True
# Opening JSON file
f = open(albet_tokenizer_model+ "_special2/tokenizer_config.json")
   
# returns JSON object as 
# a dictionary
tokenizer_config = json.load(f)

tokenizer_config['max_len'] = 512
tokenizer_config['model_type'] = 'albert'
tokenizer_config['do_lower_case'] = False
tokenizer_config['keep_accents'] = True

with open(albet_tokenizer_model+ "_special2/tokenizer_config.json", 'w') as outfile:
    json.dump(tokenizer_config, outfile)
'''
tokenizer_config = {
	"max_len": 512,
	"model_type": "albert",
	"do_lower_case":False, 
	"keep_accents":True
}
with open(albet_tokenizer_model+ "_special/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)
'''

'\ntokenizer_config = {\n\t"max_len": 512,\n\t"model_type": "albert",\n\t"do_lower_case":False, \n\t"keep_accents":True\n}\nwith open(albet_tokenizer_model+ "_special/tokenizer_config.json", \'w\') as fp:\n    json.dump(tokenizer_config, fp)\n'

In [ ]:

#To train from scratch
!python /content/transformers/examples/pytorch/language-modeling/run_mlm.py \
        --model_type albert-xlarge \
        --config_name /content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model_special2/ \
        --tokenizer_name /content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model_special2/ \
        --train_file /content/drive/MyDrive/Tokenizer_train/data/train_tokenizer.txt \
        --output_dir /content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model_special2/model \
        --do_train \
        --line_by_line \
        --save_steps 500 \
        --logging_steps 500 \
        --save_total_limit 2 \
        --num_train_epochs 1 \
        --seed 108 \
        --logging_dir /content/drive/MyDrive/Tokenizer_train/logs

2021-05-16 08:19:16.230075: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/16/2021 08:19:19 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
05/16/2021 08:19:19 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=/content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model_special2/model, overwrite_output_dir=False, do_train=True, do_eval=False, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=/content/drive/MyDrive/Tokenizer_

In [21]:
from transformers import AlbertTokenizerFast, AlbertModel

tokenizer = AlbertTokenizerFast.from_pretrained('/content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model_special2/checkpoint-1000')
model = AlbertModel.from_pretrained('/content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model_special2/checkpoint-1000')

Some weights of the model checkpoint at /content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model_special2/checkpoint-1000 were not used when initializing AlbertModel: ['predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertModel were not initialized from the model checkpoint at /content/drive/MyDrive/Tokenizer_train/albert_tokenizer_model_special2/checkpoint-1000 and are new

In [23]:
tokenizer.save_pretrained('/content/drive/MyDrive/Tokenizer_train/albert-xlarge-kor')
model.save_pretrained('/content/drive/MyDrive/Tokenizer_train/albert-xlarge-kor')

# 기존 albert-kor-base 사용 

https://huggingface.co/kykim/albert-kor-base

In [5]:
from transformers import BertTokenizerFast, AlbertModel

tokenizer = BertTokenizerFast.from_pretrained('kykim/albert-kor-base')
model = AlbertModel.from_pretrained('kykim/albert-kor-base')

Some weights of the model checkpoint at kykim/albert-kor-base were not used when initializing AlbertModel: ['predictions.dense.weight', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.decoder.bias', 'predictions.decoder.weight', 'sop_classifier.classifier.bias', 'sop_classifier.classifier.weight', 'predictions.dense.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
tokenizer.save_pretrained('/content/drive/MyDrive/Tokenizer_train/albert-kor-base')

('/content/drive/MyDrive/Tokenizer_train/albert-kor-base/tokenizer_config.json',
 '/content/drive/MyDrive/Tokenizer_train/albert-kor-base/special_tokens_map.json',
 '/content/drive/MyDrive/Tokenizer_train/albert-kor-base/vocab.txt',
 '/content/drive/MyDrive/Tokenizer_train/albert-kor-base/added_tokens.json',
 '/content/drive/MyDrive/Tokenizer_train/albert-kor-base/tokenizer.json')

In [7]:
model.save_pretrained('/content/drive/MyDrive/Tokenizer_train/albert-kor-base')

In [4]:
from transformers import AlbertTokenizer, AlbertModel

tokenizer = AlbertTokenizer.from_pretrained('albert-xlarge-v2')
model = AlbertModel.from_pretrained('albert-xlarge-v2')

Some weights of the model checkpoint at albert-xlarge-v2 were not used when initializing AlbertModel: ['predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
tokenizer.save_pretrained('/content/drive/MyDrive/Tokenizer_train/albert-kor-xlarge')
model.save_pretrained('/content/drive/MyDrive/Tokenizer_train/albert-kor-xlarge')

# 여기서부터 새로운 model

In [4]:
from transformers import AlbertTokenizerFast, AlbertForQuestionAnswering

tokenizer = AlbertTokenizerFast.from_pretrained('/content/drive/MyDrive/Tokenizer_train/albert-xlarge-kor')
#model = AlbertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/Tokenizer_train/albert-kor-base')

In [5]:
op = tokenizer("나는 오늘 학교에 간다.", return_tensors="pt")
print(op)
print("Tokens (str)      : {}".format([tokenizer.convert_ids_to_tokens(s) for s in op['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(op['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(op['attention_mask'].tolist()[0]))

{'input_ids': tensor([[    5,   168,   459,   397,   707,   473,  1823, 17709,     6]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Tokens (str)      : ['[CLS]', '▁나', '는', '▁오늘', '▁학교', '에', '▁간다', '.', '[SEP]']
Tokens (int)      : [5, 168, 459, 397, 707, 473, 1823, 17709, 6]
Tokens (attn_mask): [1, 1, 1, 1, 1, 1, 1, 1, 1]



# KoQuAD1.0 데이터 Load

In [6]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric, load_from_disk

squad_dataset = load_dataset('squad_kor_v1') 

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/92f88eedc7d67b3f38389e8682eabe68caa450442cc4f7370a27873dbc045fe4. Subsequent calls will reuse this data.


In [7]:
%%time

from datasets import Dataset
import pandas as pd

squad_dataset_train = {}
squad_dataset_train['id'] = []
squad_dataset_train['title'] = []
squad_dataset_train['context'] = []
squad_dataset_train['question'] = []
squad_dataset_train['answers'] = []


for sdt in squad_dataset['train']:
    if  len(sdt['answers']['answer_start']) > 0:
        squad_dataset_train['id'].append(sdt['id'])
        squad_dataset_train['title'].append(sdt['title'])
        squad_dataset_train['context'].append(sdt['context'])
        squad_dataset_train['question'].append(sdt['question'])
        squad_dataset_train['answers'].append(sdt['answers'])

dataset_train = Dataset.from_dict(squad_dataset_train)

squad_dataset_val = {}
squad_dataset_val['id'] = []
squad_dataset_val['title'] = []
squad_dataset_val['context'] = []
squad_dataset_val['question'] = []
squad_dataset_val['answers'] = []

for sdt in squad_dataset['validation']:
    if  len(sdt['answers']['answer_start']) > 0:
        squad_dataset_val['id'].append(sdt['id'])
        squad_dataset_val['title'].append(sdt['title'])
        squad_dataset_val['context'].append(sdt['context'])
        squad_dataset_val['question'].append(sdt['question'])
        squad_dataset_val['answers'].append(sdt['answers'])

dataset_val = Dataset.from_dict(squad_dataset_val)

CPU times: user 9.25 s, sys: 463 ms, total: 9.71 s
Wall time: 9.38 s


In [8]:
%%time

train_contexts = dataset_train['context']
train_questions = dataset_train['question']
train_answers = dataset_train['answers']

val_contexts = dataset_val['context']
val_questions = dataset_val['question']
val_answers = dataset_val['answers']

CPU times: user 1.28 s, sys: 79.4 ms, total: 1.35 s
Wall time: 1.35 s


In [9]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        try:
        
            gold_text = answer['text']
            if isinstance(gold_text,list):
                gold_text = gold_text[0]

            start_idx = answer['answer_start']
            if isinstance(start_idx,list):
                start_idx = start_idx[0]

            end_idx = start_idx + len(gold_text)

            # sometimes squad answers are off by a character or two – fix this
            if context[start_idx:end_idx] == gold_text:
                answer['answer_end'] = [end_idx]
            elif context[start_idx-1:end_idx-1] == gold_text:
                answer['answer_start'] = [start_idx - 1]
                answer['answer_end'] = [end_idx - 1]     # When the gold label is off by one character
            elif context[start_idx-2:end_idx-2] == gold_text:
                answer['answer_start'] = [start_idx - 2]
                answer['answer_end'] = [end_idx - 2]     # When the gold label is off by two characters
        except Exception as e:
            pass
            #print('context',context)
            #print('gold_text',gold_text)


add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [10]:

train_encodings = tokenizer(train_contexts, train_questions, max_length=512, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, max_length=512, truncation=True, padding=True)

In [11]:
squad_dataset = None

In [12]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start'][0]))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'][0] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

# 학습

In [13]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    #def __getitem__(self, idx):
    #    return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [17]:
torch.cuda.empty_cache()
gc.collect()

0

In [14]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = AlbertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/Tokenizer_train/albert-xlarge-kor')
model.to(device)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
optim = AdamW(model.parameters(), lr=5e-5)

Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at /content/drive/MyDrive/Tokenizer_train/albert-xlarge-kor and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Print iterations progress
class ProgressBar:

  def __init__(self,total=20, prefix = '', suffix = '', decimals = 1, length = 20, fill = '█', printEnd = "\r"):
    self.total = total
    self.prefix = prefix
    self.suffix = suffix
    self.decimals = decimals
    self.length = length
    self.fill = fill
    self.printEnd = printEnd
    self.ite = 0

  def printProgress(self,iteration, text):
      self.ite += iteration
      percent = ("{0:." + str(self.decimals) + "f}").format(100 * (self.ite / float(self.total)))

      filledLength = int(self.length * self.ite // self.total)
      bar = self.fill * filledLength + '-' * (self.length - filledLength)
      print(f'\r{self.prefix} |{bar}| {percent}% {self.suffix}  {text}', end="", flush=True)
      # Print New Line on Complete
      if self.ite == self.total: 
          print()

In [16]:

batch_count = len(train_loader)
model.to(device)
model.train()
#loss_graph = []
epochs = 1

for epoch in range(epochs):
    #seeding(1024+epoch)
    pb = ProgressBar(total=batch_count,prefix='Epoch '+ str(epoch+1) + '/' + str(epochs))
    current_batch = 0
    for batch in train_loader:
        current_batch+=1
        #seeding(1024+epoch+current_batch)
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        #print('input_ids',len(input_ids))
        attention_mask = batch['attention_mask'].to(device)
        #print('attention_mask',len(attention_mask))
        start_positions = batch['start_positions'].to(device)
        #print('start_positions',len(start_positions))
        end_positions = batch['end_positions'].to(device)
        #print('end_positions',len(end_positions))
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
        pb.printProgress(+1,'loss:' + str(loss))
        #loss_graph.append(loss.item())
    model.save_pretrained('/content/drive/MyDrive/korQuAD1.0/model')

model.eval()

Epoch 1/1 |--------------------| 0.3%   loss:tensor(6.2383, device='cuda:0', grad_fn=<DivBackward0>)

KeyboardInterrupt: ignored

In [ ]:

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/korQuAD1.0/model',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/content/drive/MyDrive/korQuAD1.0/logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

In [29]:
model = AlbertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/korQuAD1.0/model')
#model = AlbertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/Tokenizer_train/albert-kor-base')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

examples = dataset_val

def display_example(example):    
    from pprint import pprint

    #idx = qid_to_example_index[qid]
    q = example['question']
    c = example['context']
    a = example['answers']['text']
    
    print(f'Example {example["id"]} of {len(examples)}\n---------------------')
    print(f"Q: {q}\n")
    print("Context:")
    pprint(c)
    print(f"\nTrue Answers:\n{a}")

from pprint import pprint

def get_prediction(example,display=False):

    if display:
        display_example(example)
    # given a question id (qas_id or qid), load the example, get the model outputs and generate an answer
    question = example['question'] #examples[qid_to_example_index[qid]].question_text
    context = example['context'] #examples[qid_to_example_index[qid]].context_text

    #print(f"Q: {question}\n")
    #print("Context:")
    #pprint(context)

    inputs = tokenizer.encode_plus(question, context, max_length=256,return_tensors='pt')

    inputs.to(device)
    
    outputs = model(**inputs)

    #print(outputs)
    #print(outputs)    
    answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(outputs[1]) + 1 

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    if display:
        print('')
        print('Predict answer:',answer)
    return answer

 # these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def get_gold_answers(example):
    """helper function that retrieves all possible true answers from a squad2.0 example"""
    
    gold_answers = example['answers']['text'] #[answer["text"] for answer in example['answers'] if answer["text"]]

    # if gold_answers doesn't exist it's because this is a negative example - 
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]
        
    return gold_answers

def evaluate_example(example,display=False):
    gold_answers = get_gold_answers(example)
    prediction = get_prediction(example)
    em_score = max((compute_exact_match(prediction, answer)) for answer in gold_answers)
    f1_score = max((compute_f1(prediction, answer)) for answer in gold_answers)
    if display:
        print(f"Question: {example['question']}")
        print(f"Prediction: {prediction}")
        print(f"True Answers: {gold_answers}")
        print(f"EM: {em_score} \t F1: {f1_score}")
    return em_score,f1_score

In [30]:
em,f1 = evaluate_example(examples[0],display=True)

Question: 임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?
Prediction: 1989년 2월 15일
True Answers: ['1989년 2월 15일']
EM: 1 	 F1: 1.0


In [31]:
import numpy as np

ems = []
f1s = []
pb = ProgressBar(total=len(examples),prefix='Evaluate... ')
for example in examples:
    em,f1 = evaluate_example(example)
    ems.append(em)
    f1s.append(f1)
    pb.printProgress(+1,'EM:' + str(em) + '  F1:' + str(f1))
print('EM:',np.mean(ems))
print('F1:',np.mean(f1s))

Evaluate...  |████████████████████| 100.0%   EM:0  F1:0.5
EM: 0.4664011084170419
F1: 0.5565387760024785
